In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with Cloud API Registry on Vertex AI Agent Engine

## Overview

This tutorial shows you how to discover and list available MCP servers and tools using **Cloud API Registry**


### What you'll learn

- How to discover and list available MCP servers and tools using Cloud API Registry
- How to enable BigQuery MCP server tools for your project


### Prerequisites

- A Google Cloud project with billing enabled
- Basic familiarity with Python and BigQuery (helpful but not required)

## Get started

### Install required packages

**Note:** This installation will take a few moments. After it completes, you may see a message prompting you to restart the runtime. This is expected behavior—please click the **"Restart Runtime"** button when prompted.

In [ ]:
# Install Vertex AI SDK with Agent Engine and ADK support
%pip install "google-cloud-aiplatform[agent_engines,adk]>=1.101.0" --quiet

### Authenticate your notebook environment (Colab only)

If you're running this notebook in Google Colab, run the cell below to authenticate your Google Cloud account. This allows the notebook to access your Cloud resources.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    print("✅ Authentication successful!")

### Set Google Cloud project information

To use Vertex AI and Cloud API Registry, you need to specify your Google Cloud project details. Replace the placeholder values below with your actual project ID and preferred location.

We'll also create a Cloud Storage bucket to store artifacts during agent deployment.

In [ ]:
import os

import vertexai

# Replace with your project information
# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on

# Use environment variable if not explicitly set
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

# Set location (use us-central1 for best availability)
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

# Create a unique bucket name for this tutorial
import uuid

BUCKET_NAME = f"{PROJECT_ID}-tool-registry-{uuid.uuid4().hex[:8]}"
BUCKET_URI = f"gs://{BUCKET_NAME}"

print(f"📋 Project ID: {PROJECT_ID}")
print(f"📍 Location: {LOCATION}")
print(f"🪣 Bucket: {BUCKET_URI}")

### Create Cloud Storage bucket

This bucket will store temporary files during agent deployment.

In [ ]:
# Create the bucket
!gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}
print("\n✅ Bucket created successfully!")

### Enable required Google Cloud APIs

**Why we need these APIs:**
- **apihub.googleapis.com**: Provides access to API Hub for discovering MCP servers
- **cloudapiregistry.googleapis.com**: The Cloud API Registry service that manages MCP tools
- **aiplatform.googleapis.com**: Vertex AI for deploying and running agents

**Note:** This command may take 1-2 minutes to complete. This is normal.

In [ ]:
# Enable the required APIs
!gcloud services enable apihub.googleapis.com cloudapiregistry.googleapis.com aiplatform.googleapis.com --project={PROJECT_ID}
print("\n✅ All required APIs enabled successfully!")

### Initialize Vertex AI SDK and ADK

Now we'll initialize the Vertex AI SDK and set environment variables required by the Agent Development Kit (ADK).

In [ ]:
# Set ADK environment variables
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

print("✅ Vertex AI and ADK initialized successfully!")

## Discover MCP Tools with Cloud API Registry

Cloud API Registry acts as a centralized catalog of MCP (Model Context Protocol) servers. Each MCP server exposes a set of **tools** that agents can use to perform specific tasks.

In this section, you'll:
1. List all available MCP servers (both enabled and disabled)
2. Filter for the BigQuery MCP server
3. Examine what tools the BigQuery server provides

Let's start by discovering what MCP servers are available in the registry.

### List all available MCP servers

The `--all` flag shows both enabled and disabled servers. Without it, you'd only see servers you've already enabled for your project.

In [ ]:
# List all MCP servers available in the registry
!gcloud beta api-registry mcp servers list --all --project={PROJECT_ID}

### Find the BigQuery MCP server

Now let's filter specifically for the BigQuery server by its URL. This shows us the server's status (enabled/disabled) and endpoint information.

In [ ]:
# Filter for BigQuery MCP server
!gcloud beta api-registry mcp servers list --all \
    --filter='urls="bigquery.googleapis.com/mcp"' \
    --project={PROJECT_ID}

print("💡 Note: If the state shows 'DISABLED', we'll enable it in the next section.")

### List available BigQuery MCP tools

Each MCP server provides specific tools. Let's see what capabilities the BigQuery server offers.

**Note:** This command lists tools even if the server isn't enabled yet. We're just exploring what's available.

In [ ]:
# List all tools provided by the BigQuery MCP server
!gcloud beta api-registry mcp tools list --all \
    --filter='mcpServerUrls="bigquery.googleapis.com/mcp"' \
    --project={PROJECT_ID}

### Understanding BigQuery MCP Tools

The BigQuery MCP server provides five powerful tools for working with your data:

| Tool Name | Description | Use Case |
|-----------|-------------|----------|
| `list_dataset_ids` | Lists all dataset IDs in a project | Discovering what datasets are available |
| `get_dataset_info` | Retrieves metadata about a specific dataset | Understanding dataset properties and settings |
| `list_table_ids` | Lists all table IDs within a dataset | Exploring what tables exist in a dataset |
| `get_table_info` | Retrieves schema and metadata for a table | Examining column names, types, and structure |
| `execute_sql` | Runs a SQL query and returns results | Analyzing data with SELECT statements |

**Important notes about `execute_sql`:**
- ✅ Only `SELECT` statements are allowed
- ❌ `INSERT`, `UPDATE`, `DELETE`, and stored procedures are blocked
- All queries are automatically labeled with `goog-mcp-server: true` for tracking

### Enable BigQuery MCP Server

Before we can use the BigQuery tools in our agent, we need to enable the MCP server for our project. This grants our project permission to use the server's tools.

**What happens when you enable a server:**
- The server becomes available for agents in your project
- Your project's agents can call the server's tools
- The tools use your project's credentials and quotas

In [ ]:
# Enable the BigQuery MCP server
!gcloud beta api-registry mcp enable bigquery.googleapis.com --project={PROJECT_ID}

print("\n✅ BigQuery MCP server enabled successfully!")
print("\nYour agents can now use BigQuery tools to:")
print("  • List and explore datasets")
print("  • Examine table schemas")
print("  • Execute SQL queries")

### Verify the server is enabled

Let's confirm the BigQuery server now shows as `ENABLED` in our project.

In [ ]:
# List only enabled servers (should now include BigQuery)
!gcloud beta api-registry mcp servers list --project={PROJECT_ID}

print("\n✅ The BigQuery server should now appear with state: ENABLED")

### Grant IAM permissions for MCP Tool User role

This allows your user account to call MCP tools through the API Registry

In [ ]:
user_email = !gcloud config get-value account
member = f"user:{user_email[0]}"

!gcloud projects add-iam-policy-binding {PROJECT_ID} --member={member} --role="roles/mcp.toolUser" --quiet --condition=None
print("\n✅ IAM permissions granted!")

## Next Steps

Congratulations! You've successfully:
- Discovered MCP servers using Cloud API Registry
- Enabled BigQuery MCP tools


### Learn more:

- [Cloud API Registry documentation](https://docs.cloud.google.com/api-registry/docs/overview)
- [Vertex AI Agent Engine documentation](cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)
- [Google Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- [Model Context Protocol](https://modelcontextprotocol.io/)